### Required Libraries

In [ ]:
pip install torch transformers pandas scikit-learn

In [1]:
import torch

# Device Configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)


Using device: cuda


In [2]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [3]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

True
2
Tesla T4


In [4]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [5]:
import pandas as pd
import numpy as np
import torch
import re
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [6]:
df_fake = pd.read_csv('/kaggle/input/fake-news-detection/fake.csv')
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [7]:
df_real = pd.read_csv('/kaggle/input/fake-news-detection/true.csv')
df_real.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [8]:
# Modify dataset preparation
def prepare_dataset():
    df_fake['label'] = 0  # Use numeric labels
    df_real['label'] = 1  # Use numeric labels
    df = pd.concat([df_fake, df_real], ignore_index=True)  # Reset index
    df.to_csv('/kaggle/working/combined_dataset.csv', index=False)
    return df

In [9]:
df = prepare_dataset()

In [10]:
print(f'Data rows: {df.shape[0]}, Data columns: {df.shape[1]}')

Data rows: 44898, Data columns: 5


In [11]:
df.isna().sum()

title      0
text       0
subject    0
date       0
label      0
dtype: int64

In [12]:
df.duplicated().sum()

209

In [13]:
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
print(f'Data rows after cleaning: {df.shape[0]}, Data columns after cleaning: {df.shape[1]}')


Data rows after cleaning: 44689, Data columns after cleaning: 5


In [14]:
# Data Preprocessing (Similar to original EDA)
def preprocess_text(text):
    # Add your text cleaning logic here
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text

In [15]:
# Modify Dataset class
class FakeNewsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=512):
        self.texts = texts.reset_index(drop=True)  # Reset index
        self.labels = labels.reset_index(drop=True)  # Reset index
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])  # Use iloc
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_tensors='pt'
        )
        
        print(f"Input ID Shape: {inputs['input_ids'].shape}, Label: {self.labels[idx]}")  # Debug
        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'labels': torch.tensor(self.labels.iloc[idx], dtype=torch.long)
        }

In [16]:
import torch
from torch.optim import AdamW
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import time

def train_model(model, train_loader, val_loader, device, epochs=1, accumulation_steps=4):
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=2e-5)
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        
        # Progress bar for training
        progress_bar = tqdm(enumerate(train_loader), 
                             total=len(train_loader), 
                             desc=f'Epoch {epoch+1}/{epochs}', 
                             unit='batch')
        
        epoch_start_time = time.time()
        
        for i, batch in progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            outputs = model(input_ids, 
                            attention_mask=attention_mask, 
                            labels=labels)
            
            loss = outputs.loss / accumulation_steps
            total_loss += loss.item()
            loss.backward()
            
            if (i + 1) % accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()
            
            # Update progress bar with current loss
            progress_bar.set_postfix({
                'Loss': f'{total_loss / (i+1):.4f}'
            })
        
        epoch_time = time.time() - epoch_start_time
        avg_epoch_loss = total_loss / len(train_loader)
        
        # Validation
        model.eval()
        val_preds, val_true = [], []
        with torch.no_grad():
            val_progress_bar = tqdm(val_loader, 
                                    desc='Validation', 
                                    unit='batch')
            for batch in val_progress_bar:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                
                outputs = model(input_ids, attention_mask=attention_mask)
                _, preds = torch.max(outputs.logits, dim=1)
                
                val_preds.extend(preds.cpu().numpy())
                val_true.extend(labels.cpu().numpy())
        
        val_accuracy = accuracy_score(val_true, val_preds)
        
        # Detailed logging
        print(f'Epoch {epoch+1} Summary:')
        print(f'  Training Time: {epoch_time:.2f} seconds')
        print(f'  Average Training Loss: {avg_epoch_loss:.4f}')
        print(f'  Validation Accuracy: {val_accuracy:.4f}')
    
    return model


In [17]:
df = pd.read_csv('/kaggle/working/combined_dataset.csv')
    
# Preprocessing
df['cleaned_text'] = df['text'].apply(preprocess_text)
    
# Split data
X_train, X_val, y_train, y_val = train_test_split(
        df['cleaned_text'], df['label'], test_size=0.2
    )
    
# Initialize BERT Tokenizer and Model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#model = BertForSequenceClassification.from_pretrained('prajjwal1/bert-tiny', num_labels=2)
model = BertForSequenceClassification.from_pretrained(
        'bert-base-uncased', num_labels=2  # Binary classification
    )
    
# Create Datasets
train_dataset = FakeNewsDataset(X_train, y_train, tokenizer)
val_dataset = FakeNewsDataset(X_val, y_val, tokenizer)
    
# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
    
# Device Configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
# Train Model
trained_model = train_model(model, train_loader, val_loader, device)
    
# Evaluation
model.eval()
y_pred = []
with torch.no_grad():
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs.logits, dim=1)
        y_pred.extend(preds.cpu().numpy())
    
print(classification_report(y_val, y_pred))

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

...

Input ID Shape: torch.Size([1, 512]), Label: 0
Input ID Shape: torch.Size([1, 512]), Label: 1
Input ID Shape: torch.Size([1, 512]), Label: 1
Input ID Shape: torch.Size([1, 512]), Label: 1
Input ID Shape: torch.Size([1, 512]), Label: 1
Input ID Shape: torch.Size([1, 512]), Label: 1
Input ID Shape: torch.Size([1, 512]), Label: 0
Input ID Shape: torch.Size([1, 512]), Label: 1
Input ID Shape: torch.Size([1, 512]), Label: 0
Input ID Shape: torch.Size([1, 512]), Label: 0
Input ID Shape: torch.Size([1, 512]), Label: 0
Input ID Shape: torch.Size([1, 512]), Label: 0
Input ID Shape: torch.Size([1, 512]), Label: 0
Input ID Shape: torch.Size([1, 512]), Label: 0
Input ID Shape: torch.Size([1, 512]), Label: 0
Input ID Shape: torch.Size([1, 512]), Label: 1


Validation: 100%|█████████▉| 560/562 [06:26<00:01,  1.49batch/s]

Input ID Shape: torch.Size([1, 512]), Label: 1
Input ID Shape: torch.Size([1, 512]), Label: 0
Input ID Shape: torch.Size([1, 512]), Label: 1
Input ID Shape: torch.Size([1, 512]), Label: 1
Input ID Shape: torch.Size([1, 512]), Label: 1
Input ID Shape: torch.Size([1, 512]), Label: 0
Input ID Shape: torch.Size([1, 512]), Label: 0
Input ID Shape: torch.Size([1, 512]), Label: 1
Input ID Shape: torch.Size([1, 512]), Label: 0
Input ID Shape: torch.Size([1, 512]), Label: 1
Input ID Shape: torch.Size([1, 512]), Label: 0
Input ID Shape: torch.Size([1, 512]), Label: 0
Input ID Shape: torch.Size([1, 512]), Label: 1
Input ID Shape: torch.Size([1, 512]), Label: 0
Input ID Shape: torch.Size([1, 512]), Label: 1
Input ID Shape: torch.Size([1, 512]), Label: 1


Validation: 100%|██████████| 562/562 [06:26<00:00,  1.45batch/s]


Input ID Shape: torch.Size([1, 512]), Label: 0
Input ID Shape: torch.Size([1, 512]), Label: 0
Input ID Shape: torch.Size([1, 512]), Label: 1
Input ID Shape: torch.Size([1, 512]), Label: 0
Epoch 1 Summary:
  Training Time: 3728.41 seconds
  Average Training Loss: 0.0057
  Validation Accuracy: 0.9992
Input ID Shape: torch.Size([1, 512]), Label: 1
Input ID Shape: torch.Size([1, 512]), Label: 1
Input ID Shape: torch.Size([1, 512]), Label: 1
Input ID Shape: torch.Size([1, 512]), Label: 1
Input ID Shape: torch.Size([1, 512]), Label: 1
Input ID Shape: torch.Size([1, 512]), Label: 1
Input ID Shape: torch.Size([1, 512]), Label: 0
Input ID Shape: torch.Size([1, 512]), Label: 0
Input ID Shape: torch.Size([1, 512]), Label: 1
Input ID Shape: torch.Size([1, 512]), Label: 1
Input ID Shape: torch.Size([1, 512]), Label: 0
Input ID Shape: torch.Size([1, 512]), Label: 1
Input ID Shape: torch.Size([1, 512]), Label: 0
Input ID Shape: torch.Size([1, 512]), Label: 0
Input ID Shape: torch.Size([1, 512]), Labe

In [18]:
torch.save(model, 'fake_news_bert.model')

In [ ]:
fake_news_detector = model.save("/kaggle/working/fake_news_detector.model")

In [20]:
torch.save(model.state_dict(), 'Fake_news_bert.model')